In [ ]:
import warnings

# warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import copy
import json
import pandas as pd
import os
import sys
import featuretools as ft
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

sys.path.append("../../")
from Trane import trane as trane
from datetime import datetime, timedelta

### Upload of the dataset and metadata

In [ ]:
df = pd.read_csv("./chicago-bike/data/bike-sampled.csv", sep=",")
df["date"] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df = df.sort_values(by=["date"])
df = df.fillna(0)
meta = trane.TableMeta(json.loads(open("./chicago-bike/data/meta.json").read()))

### Defining entity column, time column and cutoff strategy

In [ ]:
entity = "usertype"
time = "date"
cutoff = "1h"
cutoff_base = pd.Timestamp(datetime.strptime("2017-01-02", "%Y-%m-%d"))
cutoff_end = pd.Timestamp(datetime.strptime("2017-01-31", "%Y-%m-%d"))
cutoff_strategy = trane.CutoffStrategy(entity, cutoff, cutoff_base, cutoff_end)

problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity, cutoff_strategy=cutoff_strategy, time_col=time
)

problems = problem_generator.generate(df, generate_thresholds=True)

### Generating prediction problems

In [ ]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity, time_col=time, cutoff_strategy=cutoff_strategy
)

problems = problem_generator.generate(df, generate_thresholds=True)

### Labeling the prediction tasks

In [ ]:
for p in problems:
    try:
        x = p.execute(df, -1)
        problem_label_dict[str(p)] = x
    except:
        pass